In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

data, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)

training_data, testing_data = data['train'], data['test']

name_classes = metadata.features['label'].names

def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

training_data = training_data.map(normalize)
testing_data = testing_data.map(normalize)

training_data = training_data.cache()
testing_data = testing_data.cache()

2024-10-05 18:41:50.440487: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-05 18:41:50.441247: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-05 18:41:50.445667: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-05 18:41:50.460818: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-05 18:41:50.478671: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

/home/aditya/Projects/intelligent-wardrobe/backend/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:01<00:00,  2.17 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:03<00:00,  1.17 url/s]


Dataset fashion_mnist downloaded and prepared to /home/aditya/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:

for image, label in training_data.take(1):
   break
image = image.numpy().reshape((28,28))

import matplotlib.pyplot as plt

plt.figure()
plt.imshow(image, cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()


In [ ]:
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(training_data.take(25)):
  image = image.numpy().reshape((28,28))
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image, cmap=plt.cm.binary)
  plt.xlabel(name_classes[label])
plt.show()

In [2]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(50, activation=tf.nn.relu),
    tf.keras.layers.Dense(50, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

num_training_data = metadata.splits["train"].num_examples
num_test_data = metadata.splits["test"].num_examples
BATCH = 32

training_data = training_data.repeat().shuffle(num_training_data).batch(BATCH)
testing_data = testing_data.batch(BATCH)



/home/aditya/Projects/intelligent-wardrobe/backend/venv/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
import math

res = model.fit(training_data, epochs=5, steps_per_epoch= math.ceil(num_training_data / BATCH))

plt.xlabel("# epoch")
plt.ylabel("loss")
plt.plot(res.history["loss"])



In [ ]:
import matplotlib.pyplot as plt

index = 50

for image, label in testing_data.take(index): 
    image = image[0].numpy() 
    image = image.reshape((28, 28)) 

    plt.figure()
    plt.imshow(image, cmap=plt.cm.binary)
    plt.colorbar()
    plt.grid(False)
    plt.show()   

    # Perform prediction
    predictions = model.predict(image.reshape(1, 28, 28))   
    predicted_class = tf.argmax(predictions, axis=1).numpy()[0]

    class_names = metadata.features['label'].names

    # Display prediction result
    print(f"Predicted Class: {class_names[predicted_class]}")

In [4]:
import pickle

pickle.dump(model, open("model.pkl", "wb"))